<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Подготовка-данных-для-обучения" data-toc-modified-id="Подготовка-данных-для-обучения-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных для обучения</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [2]:
#подключение необходимых библиотек
import numpy as np
import pandas as pd
import re
import nltk
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score, make_scorer
from sklearn.dummy import DummyClassifier

import warnings
warnings.filterwarnings('ignore')

In [3]:
#подключение и просмотр данных
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.sample(5)

,Unnamed: 0,text,toxic
154098,154255,"""\n\n(Belated for edit conflict) Let me try to...",0
42941,42991,"""\n\nWhat does """"Start Class"""" mean? Going to ...",0
16434,16451,"""\n\nTalk:Female genital mutilation; please st...",0
80599,80675,Molecular vibration \nThanks for the link. I h...,0
69412,69480,What are you doing?? \n\nThe info you added do...,0


In [6]:
#уберем лишние столбцы с индексами
data = data[['text', 'toxic']]

#обозначим корпус
corpus = data['text'].values

#посмотрим на соотношение ответов
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Данные не сбалансированны

### Предобработка данных

Напишем функции для лемматизации и очистки текста .

In [7]:
#функция очистки текста
def clear_text(text):
    cl_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clear_list = cl_text.split()
    clear_text = " ".join(clear_list)
    return clear_text

In [13]:

def get_wordnet_pos(text):
    tag = nltk.pos_tag([text])[0][1][0].upper()
    tag_dict ={ 'J' : wordnet.ADJ,
              "N" : wordnet.NOUN,
              "V" : wordnet.VERB,
              "R" : wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [9]:
#функция лемматизации
wnl = WordNetLemmatizer()
def lemmatize(text):
    text = text.lower()
    lemm_list = [wnl.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)] 
    #[wnl.lemmatize(w) for w in nltk.word_tokenize(text)]
    lemm_text = " ".join(lemm_list)
    return lemm_text

In [15]:
#применение
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
lemm=[]
for i in tqdm(range(len(corpus))):
    lemm.append(lemmatize(clear_text(corpus[i])))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
100%|██████████| 159292/159292 [21:33<00:00, 123.19it/s]


In [18]:
data['lemm_text'] = pd.Series(lemm, index=data.index)

In [19]:
#проверка
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


### Подготовка данных для обучения

In [20]:
#разделение на тестовую и обучающую выборки
X_train_clear, X_test_clear = train_test_split(data['lemm_text'], test_size=.25, random_state=12345)
y_train, y_test = train_test_split(data['toxic'], test_size=.25, random_state=12345)

#просмотр выборок
X_train_clear.shape, X_test_clear.shape, y_train.shape, y_test.shape

((119469,), (39823,), (119469,), (39823,))

Очистка данных от стоп-слов

In [21]:
%time
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

X_train = count_tf_idf.fit_transform(X_train_clear)
X_test = count_tf_idf.transform(X_test_clear)
X_train.shape, X_test.shape

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


((119469, 133975), (39823, 133975))

**Вывод:**
* данные подготовлены к дальнейшему обучению
* удалены стоп-слова
* произведена очистка текста от лишних символов, а так же лемматизация
* в данных наблюдается большой дисбаланс классов
* данные поделены на обучающую и тестовую выборки

## Обучение

In [22]:
#объявление констант
splits = 3
random = [12345]
score = make_scorer(f1_score)

### Логистическая регрессия

In [23]:
%%time

params_log = {"class_weight": ['balanced'], 
             "random_state": random,
             "max_iter": [100, 150, 200],
             "C": [0.1, 1.0, 10]
            }
model_log = LogisticRegression()
model_log_rs = RandomizedSearchCV(model_log, cv=splits, scoring=score,
                                  param_distributions=params_log, n_iter=splits, verbose=True)
model_log_rs.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
CPU times: user 6min, sys: 6min 46s, total: 12min 46s
Wall time: 12min 51s


RandomizedSearchCV(cv=3, estimator=LogisticRegression(), n_iter=3,
                   param_distributions={'C': [0.1, 1.0, 10],
                                        'class_weight': ['balanced'],
                                        'max_iter': [100, 150, 200],
                                        'random_state': [12345]},
                   scoring=make_scorer(f1_score), verbose=True)

In [24]:
#лучшие параметры
print('Лучший результат f1: ', model_log_rs.best_score_)
model_log_rs.best_params_

Лучший результат f1:  0.755854867297927


{'random_state': 12345, 'max_iter': 200, 'class_weight': 'balanced', 'C': 10}

**Вывод:** Лучшая метрика **F1 = 0.7594343341767904** у модели **Логистическая регрессия** при гиперпараметрах: **{'random_state': 12345, 'max_iter': 200, 'class_weight': 'balanced', 'C': 10}.**

### Случайный лес

In [27]:
%%time

params_forest = {"max_depth": [5, 10, 15], 
          "n_estimators": [30, 90, 150], 
          "random_state": random, 
          "class_weight": ['balanced']
         }
model_forest = RandomForestClassifier()
model_forest_rs = RandomizedSearchCV(model_forest, cv=splits, scoring=score, 
                                     param_distributions=params_forest)
model_forest_rs.fit(X_train, y_train)

CPU times: user 5min 35s, sys: 4.18 s, total: 5min 40s
Wall time: 5min 40s


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(),
                   param_distributions={'class_weight': ['balanced'],
                                        'max_depth': [5, 10, 15],
                                        'n_estimators': [30, 90, 150],
                                        'random_state': [12345]},
                   scoring=make_scorer(f1_score))

In [28]:
#лучшие параметры
print('Лучший результат f1: ', model_forest_rs.best_score_)
model_forest_rs.best_params_

Лучший результат f1:  0.39728478306390863


{'random_state': 12345,
 'n_estimators': 150,
 'max_depth': 15,
 'class_weight': 'balanced'}

**Вывод:** Лучшая метрика **F1 = 0.39728478306390863** у модели **Случайный лес** при гиперпараметрах: **{'random_state': 12345, 'n_estimators': 150, 'max_depth': 15, 'class_weight': 'balanced'}.**

### CatBoostClassifier

In [36]:
%%time

params_cat = {"max_depth": [1, 2, 3], 
          "n_estimators": [10, 100, 150], 
          "random_state": random, 
          "auto_class_weights": ['Balanced']
         }

model_cat = CatBoostClassifier(eval_metric='TotalF1')
model_cat_gs = model_cat.grid_search(params_cat, Pool(X_train,y_train),
                                     verbose=True, cv=splits, plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.4832592	test: 0.4777164	best: 0.4777164 (0)	total: 481ms	remaining: 4.33s
1:	learn: 0.4854958	test: 0.4805790	best: 0.4805790 (1)	total: 947ms	remaining: 3.79s
2:	learn: 0.4880427	test: 0.4834125	best: 0.4834125 (2)	total: 1.41s	remaining: 3.29s
3:	learn: 0.4880427	test: 0.4834125	best: 0.4834125 (2)	total: 1.88s	remaining: 2.82s
4:	learn: 0.4880427	test: 0.4834125	best: 0.4834125 (2)	total: 2.35s	remaining: 2.35s
5:	learn: 0.5466872	test: 0.5412988	best: 0.5412988 (5)	total: 2.84s	remaining: 1.89s
6:	learn: 0.5477861	test: 0.5422410	best: 0.5422410 (6)	total: 3.32s	remaining: 1.42s
7:	learn: 0.5477861	test: 0.5422410	best: 0.5422410 (6)	total: 3.79s	remaining: 948ms
8:	learn: 0.5477861	test: 0.5422410	best: 0.5422410 (6)	total: 4.26s	remaining: 474ms
9:	learn: 0.5479314	test: 0.5428742	best: 0.5428742 (9)	total: 4.74s	remaining: 0us

bestTest = 0.5428741586
bestIteration = 9

0:	loss: 0.5428742	best: 0.5428742 (0)	total: 46s	remaining: 6m 8s
0:	learn: 0.4832592	test: 0.477

In [41]:
#лучшие параметры
print('Лучший результат f1: ', model_cat.best_score_)
model_cat.get_all_params()

Лучший результат f1:  {'learn': {'Logloss': 0.26480701355064334, 'TotalF1': 0.904812841747268}}


{'nan_mode': 'Min',
 'eval_metric': 'TotalF1',
 'iterations': 150,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'subsample': 0.800000011920929,
 'use_best_model': False,
 'class_names': [0, 1],
 'random_seed': 12345,
 'depth': 3,
 'posterior_sampling': False,
 'border_count': 254,
 'class_weights': [1, 8.83931827545166],
 'classes_count': 0,
 'auto_class_weights': 'Balanced',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'Logloss',
 

**Вывод:** Лучшая метрика **F1 = 0.904812841747268** у модели **CatBoostClassifier**.

**Общий вывод:**
* Лучшая метрика F1 = 0.7594343341767904 у модели Логистическая регрессия
* Лучшая метрика F1 = 0.39728478306390863 у модели Случайный лес
* Лучшая метрика F1 = 0.904812841747268 у модели CatBoostClassifier

Для тестовой выборки берем модель **Логистическая регрессия** так как у нее неплохой результат при быстрой скорости обучения.

In [25]:
#тестирование
predict = model_log_rs.predict(X_test)
f1_score(y_test, predict)

0.7638617839513259

In [47]:
dummy_model = DummyClassifier(strategy='uniform', random_state=12345)
dummy_model.fit(X_train, y_train)
f1_score(y_test, dummy_model.predict(X_test))

0.16900237035804883

## Выводы

В ходе работы с данными были проведены следующие действия:
* импортирование данных
* удаление стоп-слов, лемматизация
* выявлен дисбаланс классов
* данные поделены на тестовую и обучающую выборки

Были обучены 3 модели со следующими результатами:
* Лучшая метрика F1 = 0.7594343341767904 у модели Логистическая регрессия
* Лучшая метрика F1 = 0.39728478306390863 у модели Случайный лес
* Лучшая метрика F1 = 0.904812841747268 у модели CatBoostClassifier

На тестовых данных была проанализированна модель Логистическая регрессия, которая показал F1 = 0.7676186089359743, что выше ожидаемой 0,75 и выше предсказаний Dummy.